In [ ]:
from statbank import StatbankClient
import pandas as pd
import dapla as dp
from io import StringIO

fileclient = dp.FileClient()
statclient = StatbankClient("LAST214")

In [ ]:
statclient.apidata_all("11721", include_id=True)

In [ ]:
mnr_path = "gs://ssb-staging-dapla-felles-data-delt/dapla-statbank-client-testing/11721_mnr_seksjon214.csv"
with fileclient.gcs_open(mnr_path, "rb") as mnr_fil:
    mnr = mnr_fil.read().decode("ISO-8859-1")
mnr = pd.read_csv(StringIO(mnr), sep=";", index_col=None, header=1)
mnr.replace("..", pd.NA)
#mnr

In [ ]:
desc = statclient.get_description("11721")

In [ ]:
total_cols = (len(desc.variables[0]['variabler']) +
len(desc.variables[0]['statistikkvariabler']) +
len(desc.variables[0]['null_prikk_missing']))

In [ ]:
if len(mnr.columns) != total_cols:
    for colnum in range(total_cols - len(mnr.columns)):
        mnr[f"prikkecol_{colnum+1}"] = ""
mnr

In [ ]:
#desc.transferdata_template()

In [ ]:
data = {'knrmakrohovmnd1.dat': mnr}

In [ ]:
errors = statclient.validate(data, "11721", printing=False)

In [ ]:
errors.keys()

In [ ]:
for err in [v for k,v in errors.items() if "categorycode_outside" in k][0].args[0]: print(err)

In [ ]:
for err in [v for k,v in errors.items() if "floats" in k]: print(err)

In [ ]:
for err in [v for k,v in errors.items() if "rounding" in k]: print(err)

In [ ]:
statclient.transfer(data, "11721")